左の📂を押して、EXCELファイルをドラック＆ドロップで読み込んでください

In [ ]:
##ファイル名を入力！！！
FileName = "SampleExcel.xlsx"

In [28]:
from IPython.utils import path
import openpyxl ##Excelを読み書きするため
import tqdm ##プログレスバーを表示するため
from openpyxl.chart import( ##Excelにグラフを描画するため
    ScatterChart,
    BarChart,
    Reference,
    Series,
)
from openpyxl.chart import Reference
from openpyxl.chart.axis import DateAxis
import math

"""↓↓↓↓↓↓↓↓↓↓↓↓データの読み込みエリア↓↓↓↓↓↓↓↓↓↓↓↓"""
inputData = openpyxl.load_workbook(FileName) ##ファイルを開く Open File.

blank, sample1, sample2, sample3 = [], [], [], [] #それぞれの値を読み込むための空の配列を準備

p = 0

for i in range(14, 1154+19, 19): #14から1154まで19刻みで繰り返す
  blank.append([])
  sample1.append([])
  sample2.append([])
  sample3.append([])
  for j in range(3): #3つの連続したデータをそれぞれの配列に追加
    blank[p].append(inputData.active.cell(row=(i+j), column=9).value)
    sample1[p].append(inputData.active.cell(row=(i+j), column=10).value)
    sample2[p].append(inputData.active.cell(row=(i+j), column=11).value)
    sample3[p].append(inputData.active.cell(row=(i+j), column=12).value)
  
  p += 1

# print(sample2)

"""↓↓↓↓↓↓↓↓↓↓↓↓データの操作エリア↓↓↓↓↓↓↓↓↓↓↓↓"""
inputData.create_sheet("EditedData") #EditedDataという名前のシートを作成

for i in range(len(blank)): #①各行について 「サンプルの値-blankの値」を行う
  for j in range(3):
    sample1[i][j] = sample1[i][j] - blank[i][j]
    sample2[i][j] = sample2[i][j] - blank[i][j]
    sample3[i][j] = sample3[i][j] - blank[i][j]
# print(sample3)

for i in range(len(sample1)): #② ①の値についてtriplicateで平均を算出する
  inputData["EditedData"].cell(row=1, column=2).value = "control"
  inputData["EditedData"].cell(row=1, column=3).value = "ACP1b"
  inputData["EditedData"].cell(row=1, column=4).value = "242a"
  inputData["EditedData"].cell(row=i+2, column=1).value = i
  sample1[i] = (sample1[i][0] + sample1[i][1] + sample1[i][2])/3
  inputData["EditedData"].cell(row=i+2, column=2).value = sample1[i]
  sample2[i] = (sample2[i][0] + sample2[i][1] + sample2[i][2])/3
  inputData["EditedData"].cell(row=i+2, column=3).value = sample2[i]
  sample3[i] = (sample3[i][0] + sample3[i][1] + sample3[i][2])/3
  inputData["EditedData"].cell(row=i+2, column=4).value = sample3[i]
# print(sample1)


"""↓↓↓↓↓↓↓↓↓↓↓↓グラフの描画エリア↓↓↓↓↓↓↓↓↓↓↓↓"""
graph = openpyxl.chart.LineChart() #グラフの種類
# graph.title  = "Sample" #グラフのタイトル  必要なら入力してください
graph.style  = 12 #グラフのフォントサイズ
graph.height = 10 #高さ
graph.width  = 15 #幅

graph.y_axis.title = inputData.sheetnames[0] #y軸ラベル
yValue = Reference(inputData["EditedData"],min_col = 2, min_row = 1, max_col = 4, max_row = 62)
xValue = Reference(inputData["EditedData"],min_col = 1, min_row = 2, max_row = 62)

graph.add_data(yValue, titles_from_data=True) ##y軸のデータ
graph.set_categories(xValue) ##x軸のデータ
graph.y_axis.numFmt = "0" ##縦軸のラベルの書式指定 小数第2位まで表示

inputData["EditedData"].add_chart(graph, "E2") ##E2にグラフを描画 Draw a graph at position E2

#X-Y軸(1)グラフスタイル
style = graph.series[0]
style.graphicalProperties.line.solidFill = "AAAAAA"   #グラフカラー
style.graphicalProperties.line.width = 40000          #グラフ幅
# style.smooth = False                                   #グラフの線を滑らかにする

#X-Y軸(2)グラフスタイル
style2 = graph.series[1]
style2.graphicalProperties.line.solidFill = "5B9BD5"  #グラフカラー
style2.graphicalProperties.line.width = 40000         #グラフ幅
# style2.smooth = True                                  #グラフの線を滑らかにする

style3 = graph.series[2]
style3.graphicalProperties.line.solidFill = "ED7D31"  #グラフカラー
style3.graphicalProperties.line.width = 40000         #グラフ幅
# style3.smooth = True                                  #グラフの線を滑らかにする



inputData.save("Edited_" + FileName) ##エクセルファイルを保存 Save Excel file
inputData.close() ##エクセルファイルを閉じる Close Excel file